In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

2023-06-30 13:23:34.341841: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro


'/device:GPU:0'

2023-06-30 13:23:34.341859: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-06-30 13:23:34.341862: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-06-30 13:23:34.341922: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-30 13:23:34.342117: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2023-06-30 13:29:45.811525: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-30 13:29:45.811632: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 790375892747318215
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 locality {
   bus_id: 1
 }
 incarnation: 279423333309021390
 physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
 xla_global_id: -1]

# pip install

In [ ]:
# !pip install --upgrade pymupdf
# !pip install PyPDF2
# # !pip install pdfplumber
# # !pip install langchain
# !pip install --upgrade transformers
# !pip install --upgrade pillow
# !pip install sentence_transformers
# !pip install faiss-cpu

# extract pdf_text

In [3]:
import fitz
import requests
import io
import pandas as pd
import os
import PyPDF2
import pdfplumber

In [4]:
file_path = r'/Users/yehocoh/Projects/Python/esg_project/PDF files.txt'
file = open(file_path, 'r')

# Read the contents of the file
pdfs = file.read()

lines_pdfs = pdfs.split('\n')

In [140]:
import aiohttp
import asyncio
from multiprocessing import Pool
from tqdm import tqdm

num_of_doc = 10

# Define your worker function
async def worker(session, url):
    url = url.replace("\r","")
    async with session.get(url) as response:
        file_data = await response.read()

    # Now open the file with fitz
    doc = fitz.open(stream=file_data, filetype='pdf')

    #get text
    text = " ".join([page.get_text() for page in doc])

    # Loop over each page to extract text and calculate the length
    total_text_len = 0
    for page in doc:
        total_text_len += len(page.get_text())

    return {
        "pdf_name": url.split('/')[-1],
        "url": url,
        "page_count": doc.page_count,
        "text_len" : total_text_len,
        "table_of_content" : doc.get_toc(),
        "pdf_text" : text
    }

# Create an aiohttp session
async with aiohttp.ClientSession() as session:
    # Create a list of tasks
    tasks = [worker(session, url) for url in lines_pdfs[:num_of_doc]]
    # Use asyncio to run all the tasks
    data = await asyncio.gather(*tasks)

df = pd.DataFrame(data)


In [147]:
df.sample()

,pdf_name,url,page_count,text_len,table_of_content,pdf_text
3,2019-2U-ESG-Report.pdf,https://sust-reports-prod-v1.s3.us-east-2.amaz...,10,19370,[],"\n \nEdtech with a human touch. \n2U, INC. ..."


In [73]:
import textwrap
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import pdfplumber

def chunk_text(text, chunk_size):
    # Split the text by paragraph and chunk it
    chunks = text.split('\n')
    return [chunks[i:i + chunk_size] for i in range(0, len(chunks), chunk_size)]

def createEmbeddings(chunked_docs):
    chunked_docs_combined = ['\n'.join(chunked_docs) for chunked_docs in chunked_docs]
    chunked_docs_combined_txt = '\n\n'.join(chunked_docs_combined)
    # Create embeddings and store them in a FAISS vector store
    embedder = HuggingFaceEmbeddings()
    vector_store = FAISS.from_texts(chunked_docs_combined_txt, embedder)
    return vector_store

# model

In [136]:
from langchain import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

def loadLLMModel():
    # llm = HuggingFacePipeline.from_model_id(model_id="declare-lab/flan-alpaca-gpt4-xl", task="text2text-generation", model_kwargs={"temperature":0, "max_length":2000})
    # llm = HuggingFacePipeline.from_model_id(model_id="declare-lab/flan-alpaca-base", task="text2text-generation", model_kwargs={"temperature":0, "max_length":2000}) #working
    # llm = HuggingFacePipeline.from_model_id(model_id="declare-lab/flan-alpaca-xxl", task="text2text-generation", model_kwargs={"temperature":0, "max_length":2000})
    # llm = HuggingFacePipeline.from_model_id(model_id="declare-lab/flan-alpaca-large", task="text2text-generation", model_kwargs={"temperature":0, "max_length":2000}) #working
    llm = HuggingFacePipeline.from_model_id(model_id="google/flan-t5-base", task="text2text-generation", model_kwargs={"temperature":0.5, "max_length":2000}) #working


    chain = load_qa_chain(llm, chain_type="stuff")
    return chain

In [138]:
chain = loadLLMModel()

In [121]:
def askQuestions(vector_store, chain, question):
    # Ask a question using the QA chain
    similar_docs = vector_store.similarity_search(question)
    response = chain.run(input_documents=similar_docs, question=question)
    return response

# class

In [139]:
def class_row_in_text(num_doc, chunk_size= 7) :

      chunked_docs = chunk_text(df["pdf_text"][num_doc], chunk_size)
      vector_store = createEmbeddings(chunked_docs)
      
      chunked_docs_combined = ['\n'.join(chunked_docs) for chunked_docs in chunked_docs]
      chunked_docs_combined_txt = '\n\n'.join(chunked_docs_combined)
      
      responses =[]
      for row in chunked_docs_combined :
          response = askQuestions(vector_store,
                                      chain,
                                            f"""For the follow sentence please classify it into just one of the following 5 categories based on its ESG related content.
                                            
                                            -'Quantitative Evidence': The sentence contains quantitative evidence that the company already has been taking actions to improve its ESG measures and already implemented the plans. Future plans do NOT count!
                                            -'Action Evidence': The sentence contains hard evidence that is not quantitative that the company already has been taking actions to improve its ESG measures and already implemented the plans. Future plans do NOT count! .
                                            -'Suggestive Evidence': The sentence doesn't contain hard evidence of taking actions to improve the company's ESG measures, but it suggests that such actions have been taken already and have been implemented. Future plans do NOT count!
                                            -'Intention': The sentence mentions a company's plans, intentions or commitments of taking actions in the future, without evidence of any actual actions yet. It include just future plans of the company to take actions related to ESG.
                                            -'Other': The sentence does not match any category.

                                            sentence : "{row}"
                                            
                                            """
                                 )
          responses.append([num_doc, response, row])
      return responses

In [141]:
all_responses = []
for i in df.index :
    responses = class_row_in_text(i)
    all_responses = all_responses + responses


/opt/homebrew/Caskroom/miniconda/base/envs/DS_env/lib/python3.11/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (518 > 512). Running this sequence through the model will result in indexing errors


In [143]:
# Convert the list of lists to a DataFrame
all_responses_df = pd.DataFrame(all_responses, columns=['num_doc', 'response', 'row'])

all_responses_df

,num_doc,response,row
0,0,Other,\n \n \n \nQUARTERLY COMPLIANCE REPORT ON COR...
1,0,Other,Name of the Company: Firstsource Solutions Li...
2,0,Other,Status \nYes/No \nRemarks \n1 \n2 \n3 \n4
3,0,Other,I. Board of Directors \n49 (I) \n \n \n(A) Com...
4,0,Other,\n(B)Non-executive \nDirectors’ \ncompensatio...
...,...,...,...
4610,9,Action Evidence,issue identified by ISOS. Integrity through le...
4611,9,Other,Report. \nSTATEMENT OF INDEPENDENCE\nISOS Gr...
4612,9,Action Evidence,they been associated with 3M’s sustainability ...
4613,9,Other,including a statement of competencies can be f...


In [144]:
all_responses_df["response"].value_counts()

response
Action Evidence             2342
Other                       1873
Quantitative Evidence        390
Optional Evidence              3
Suggestive Evidence            1
Third Party Audits             1
European                       1
Accuracy                       1
Board                          1
Assistance                     1
Public Policy Engagement       1
Name: count, dtype: int64

In [146]:
all_responses_df.to_csv("all_responses_df.csv")

In [113]:
print(df["pdf_text"][1])


47
Annual Report 2017-18
Business Responsibility Report
INTRODUCTION
The Securities and Exchange Board of India (SEBI) in 2012 mandated 
the top 100, and later in 2015 the top 500 listed entities on National 
Stock Exchange of India Limited and BSE Limited to prepare a 
‘Business Responsibility Report’ as part of the Annual Report. This 
is as per clause (f) of sub regulation (2) of Regulation 34 of SEBI 
(Listing Obligations and Disclosure Requirements) Regulations, 
2015. The report outlines the organization’s performance from the 
environmental, social and governance perspective.
Firstsource Solutions Limited (“the Company”) being part of the 
top 500 listed entities has developed this Business Responsibility 
Report based on the suggested framework of SEBI, strengthening 
its commitment towards transparent disclosure of its environmental 
and social performance.
Continuing on the Company’s mission to tackle social issues, 
the Company is committed to monitor and report its social a

In [80]:
chunked_docs = chunk_text(wrapped_text, chunk_size= 10)

chunked_docs_combined = ['\n'.join(chunked_docs) for chunked_docs in chunked_docs]

chunked_docs_combined_txt = '\n\n'.join(chunked_docs_combined)

print(chunked_docs_combined_txt)


47
Annual Report 2017-18
Business Responsibility Report
INTRODUCTION
The Securities and Exchange Board of India (SEBI) in 2012 mandated
the top 100, and later in 2015 the top 500 listed entities on National
Stock Exchange of India Limited and BSE Limited to prepare a
‘Business Responsibility Report’ as part of the Annual Report. This
is as per clause (f) of sub regulation (2) of Regulation 34 of SEBI
(Listing Obligations and Disclosure Requirements) Regulations,

2015. The report outlines the organization’s performance from the
environmental, social and governance perspective.
Firstsource Solutions Limited (“the Company”) being part of the
top 500 listed entities has developed this Business Responsibility
Report based on the suggested framework of SEBI, strengthening
its commitment towards transparent disclosure of its environmental
and social performance.
Continuing on the Company’s mission to tackle social issues,
the Company is committed to monitor and report its social and
environm

In [87]:
vector_store = createEmbeddings(chunked_docs)
print(vector_store)

In [149]:
sentences = [
    "Our PCI target for 2028 is 71 g CO2e/MJ, a > 5% decrease from 2016.",
    "In collaboration with Pavilion Energy and QatarEnergy, we jointly developed a liquefied natural gas (LNG) carbonfootprinting methodology for delivered cargoes to help advance a standard for GHG product-level accounting.",
    "This methodology is expected to improve accuracy and build stakeholder confidence in data reliability.",
    "In early 2022, we announced a one-year pilot with Project Canary to enhance our ability to demonstrate transparency in how we are lowering methane emissions in our operations.",
    "Project Canary will use its comprehensive TrustWell Certification program to review and analyze the environmental and social performance aspects of individual wells and facilities in Colorado and Texas.",
    "The PCI metric, which includes Scope 1, 2 and 3, represents the carbon intensity across the full value chain associated with bringing products to market.",
    "Chevron's efforts to achieve our 2028 Upstream methane intensity target of 2.0 kg CO2e/boe include facility design, GHG reduction projects and exploring emerging technology to identify opportunities to further lower emissions.",
    "Our planned GHG reduction projects include opportunities to reduce venting, such as a nitrogen blanket system for the tank farm at Tengizchevroil (TCO), and to reduce flaring, which also reduces methane emissions.",
    "Our standard facility design in the Permian Basin includes methane emission controls, such as vapor recovery units at central tank batteries and pneumatic controllers that utilize compressed air instead of natural gas.",
    "We are committed to further improving methane detection and direct measurement through our global methane detection campaign, which focuses on scaling up proven and emerging detection technologies and modes of deployment, such as satellites, aircraft and drones.",
    "To date, we have completed campaigns in Argentina, the Denver-Julesburg Basin, the Gulf of Mexico, the Permian Basin and TCO.",
    "We are active participants in multiple partnerships and associations focused on methane emissions.",
    "Chevron is a founding member of the Collaboratory to Advance Methane Science (CAMS), a joint industry project to conduct peer-reviewed research around methane emissions.",
    "Recent CAMS projects include an aerial survey to understand sources in the Permian Basin and the first measurement study of methane emissions from LNG transport activities.",
    "Chevron is also a founding partner of the Environmental Partnership, an industry initiative aimed at accelerating the adoption of practices that reduce methane emissions.",
    "To date, companies in this initiative have conducted more than 770,000 leak detection surveys and replaced more than 27,000 pneumatic controllers with lower- or non-emitting technologies.",
    "Recently, the Environmental Partnership collaborated with several aerial survey technology providers, including the NASA Jet Propulsion Laboratory/University of Arizona, GHGSat, Bridger Photonics and Kairos Aerospace.",
    "We're investing to reduce methane emissions and flaring.",
    "Improving methane detection, rethinking facility designs, optimizing equipment and deploying new operational practices are a few examples of the projects underway to lower emissions.",
    "Completion operations flow back to permanent facilities that have equipment to capture entrained gas.",
    "Since 2011, standard facility designs have included compressed air for pneumatic controllers, which eliminate natural gas venting for that application.",
    "Standard designs for tank batteries and compressor stations include Vapor Recovery Units, which gather emissions that can be reused onsite or sold to third parties.",
    "Flaring intensity was 79% less than Permian Basin average in 2020 due to infrastructure planning to create gas-take away capacity.",
    "OurApologies for the cutoff in the previous message. Here's the continuation:",
    "Our Mid-Continent business unit has piloted eight advanced methane detection solutions since 2016 and selected an aerial laser-based methane scanning technology for broader deployment in the Permian.",
    "Aircraft-based solutions help us and other nearby operators cost-effectively screen assets for methane emissions across a wide geographic footprint in the Permian.",
    "Early in 2021, I was asked to serve as project manager for Chevron's methane detection campaign, which is tasked with deploying methane detection technology to cover 80% of our equity emissions.",
    "Our team works with business units across Chevron's asset classes to design tailored methane detection solutions to accommodate operational needs, policy concerns and regulatory protocols.",
    "We are experimenting with a variety of technologies to expand our methane detection capabilities, including deploying airborne sensors using satellites, aircraft and drones.",
    "In addition to working within Chevron's business units, we also collaborate with regulators, universities and other operators to evaluate equipment and share best practices.",
    "I'm especially proud of our work with the Gulf of Mexico business unit, where we became one of the first operators to deploy a methane detection solution offshore using drones.",
    "Offshore is especially challenging because ocean water can confuse detection technology and create false readings.",
    "In addition, platforms have layered equipment, which can make it difficult to identify the source of emissions from overhead sensors alone.",
    "To solve this challenge, we combined emissions detection equipment from our San Joaquin Valley business unit with drones used offshore and developed a new flight procedure to detect methane emissions from the platforms.",
    "After identification, the business units work to remove or mitigate the emissions source.",
    "Methane reduction is a global challenge, and there is not a one-size-fits-all solution.",
    "I am inspired by the level of expertise within our company and our culture of innovation and experimentation.",
    "Chevron endorses the World Bank's Zero Routine Flaring by 2030 initiative.",
    "We flare natural gas only when necessary for safety and operational purposes and in areas where pipelines and other alternatives for transporting gas do not exist.",
    "Our 2028 Upstream carbon intensity target includes a specific target for flaring intensity of 3.0 kg CO2e/boe, which is a 66% reduction from our 2016 baseline.",
    "Chevron endorsed the World Bank's Zero Routine Flaring by 2030 initiative, which brings together govern ments, oil companies and development institutions to cooperate to eliminate routine flaring by no later than 2030.",
    "Chevron is an active participant in the World Bank's Global Gas Flaring Reduction (GGFR) voluntary partnership.",
    "GGFR recently partnered with the Payne Institute for Public Policy at the Colorado School of Mines to develop a transparent web platform to support real-time mapping and tracking of global gas flaring data.",
    "Chevron supported this partnership through our membership in the Oil and Gas Climate Initiative.",
    "Future progress will require applying our world-class capabilities as we aim to deliver higher returns in a lower carbon world.",
    "Our capabilities, assets and customers are distinct advantages.",
    "Chevron Technology Ventures targets external innovation and transformational technology.",
    "The Chevron Technical Center develops and deploys technology across the entire business, including integrating low-carbon technology into our operations.",
    "In 2021, we formed Chevron New Energies, a new organization dedicated to growing hydrogen, CCUS, offsets and other emerging energies.",
    "Chevron Strategy and Sustainability continues to steward the company's long-term strategy by integrating climate change, energy transition and other sustainability themes into macroeconomic forecasting, supply-and-demand forecasting, price forecasting, portfolio modelingApologies for the cutoff again. Here's the continuation:",
    "Chevron Strategy and Sustainability continues to steward the company's long-term strategy by integrating climate change, energy transition and other sustainability themes into macroeconomic forecasting, supply-and-demand forecasting, price forecasting, portfolio modeling and competitor intelligence.",
    "Chevron and more than 10 industry partners have agreed to support large-scale deployment of carbon capture and storage (CCS) to help decarbonize industrial facilities in Houston, Texas, one of the largest concentrated sources in the United States.",
    "The organizations are considering using CCS technology at facilities that generate electricity and manufacture everyday products such as plastics, motor fuels and packaging.",
    "The collaboration could lead to capturing and storing up to 50 million metric tons of CO2 per year by 2030 and about 100 million metric tons by 2040."
]

labels = [
    "Intention",
    "Action Evidence",
    "Suggestive Evidence",
    "Intention",
    "Other",
    "Other",
    "Intention",
    "Intention",
    "Action Evidence",
    "Intention",
    "Quantitative Evidence",
    "Other",
    "Action Evidence",
    "Action Evidence",
    "Action Evidence",
    "Quantitative Evidence",
    "Other",
    "Intention",
    "Action Evidence",
    "Action Evidence",
    "Action Evidence",
    "Action Evidence",
    "Quantitative Evidence",
    "Quantitative Evidence",
    "Other",
    "Other",
    "Action Evidence",
    "Other",
    "Other",
    "Other",
    "Action Evidence",
    "Other",
    "Other",
    "Other",
    "Other",
    "Intention",
    "Other",
    "Intention",
    "Intention",
    "Action Evidence",
    "Other",
    "Other",
    "Other",
    "Other",
    "Other",
    "Other",
    "Other",
    "Intention",
    "Other",
    "Other",
    "Other",
    "Other",
    "Other",
    "Intention"
]


In [159]:
def class_row_in_text_(txt, chunk_size= 7) :

      chunked_docs = chunk_text(txt, chunk_size)
      vector_store = createEmbeddings(chunked_docs)
      
      chunked_docs_combined = ['\n'.join(chunked_docs) for chunked_docs in chunked_docs]
      chunked_docs_combined_txt = '\n\n'.join(chunked_docs_combined)
      
      response = askQuestions(vector_store,
                                      chain,
                                            f"""For the follow sentence please classify it into just one of the following 5 categories based on its ESG related content.
                                            
                                            -'Quantitative Evidence': The sentence contains quantitative evidence that the company already has been taking actions to improve its ESG measures and already implemented the plans. Future plans do NOT count!
                                            -'Action Evidence': The sentence contains hard evidence that is not quantitative that the company already has been taking actions to improve its ESG measures and already implemented the plans. Future plans do NOT count! .
                                            -'Suggestive Evidence': The sentence doesn't contain hard evidence of taking actions to improve the company's ESG measures, but it suggests that such actions have been taken already and have been implemented. Future plans do NOT count!
                                            -'Intention': The sentence mentions a company's plans, intentions or commitments of taking actions in the future, without evidence of any actual actions yet. It include just future plans of the company to take actions related to ESG.
                                            -'Other': The sentence does not match any category.

                                            sentence : "{txt}"
                                            
                                            """
                                 )
      return [txt, response]

In [160]:
all_responses_ = []
for i_ in sentences :
    responses = class_row_in_text_(i_)
    all_responses_ = all_responses_ + responses

In [162]:
all_responses_

['Our PCI target for 2028 is 71 g CO2e/MJ, a > 5% decrease from 2016.',
 'Quantitative Evidence',
 'In collaboration with Pavilion Energy and QatarEnergy, we jointly developed a liquefied natural gas (LNG) carbonfootprinting methodology for delivered cargoes to help advance a standard for GHG product-level accounting.',
 'Action Evidence',
 'This methodology is expected to improve accuracy and build stakeholder confidence in data reliability.',
 'Quantitative Evidence',
 'In early 2022, we announced a one-year pilot with Project Canary to enhance our ability to demonstrate transparency in how we are lowering methane emissions in our operations.',
 'Action Evidence',
 'Project Canary will use its comprehensive TrustWell Certification program to review and analyze the environmental and social performance aspects of individual wells and facilities in Colorado and Texas.',
 'Action Evidence',
 'The PCI metric, which includes Scope 1, 2 and 3, represents the carbon intensity across the full

In [164]:
all_responses_df_ = pd.DataFrame(all_responses_)

all_responses_df_

,0
0,"Our PCI target for 2028 is 71 g CO2e/MJ, a > 5..."
1,Quantitative Evidence
2,In collaboration with Pavilion Energy and Qata...
3,Action Evidence
4,This methodology is expected to improve accura...
...,...
103,Action Evidence
104,The organizations are considering using CCS te...
105,Action Evidence
106,The collaboration could lead to capturing and ...


In [ ]:
all_responses_df_["label"] = labels

In [ ]:
all_responses_df_